# Task 1.3

## Importazione delle librerie e definizione delle funzioni

In [1]:
import os
import pickle
import numpy as np
from scipy.io.wavfile import read
from featureextraction import extract_features
import warnings
import glob
import time
import speech_recognition as spr 
import sounddevice as sd
warnings.filterwarnings("ignore")
duration = 5 
rec_rate = 44100
error = 0
total_sample = 0.0


def determine_speaker(audio, sr):
    if str(type(audio[0])) == "<class 'numpy.int16'>": audio = audio / 32768.0
    vector   = extract_features(audio,sr)
    log_likelihood = np.zeros(len(models)) 
    for i in range(len(models)):
        gmm = models[i]  #checking with each model one by one
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
        scores = np.array(gmm.score(vector))
        log_likelihood[i] = scores.sum()
        #print(speakers[i] + 'score = \t'+str(scores.sum()))
    return np.argmax(log_likelihood)


## Caricamento dei modelli

Vengono importati i modelli dai file pickle in cui sono stati salvati dal train, dopodichè viene proposto di testarli su uno solo dei file di test o su tutti calcolando l'accuracy media

In [2]:
source   = "test_data/"   
modelpath = "gmms/"
gmm_files = [os.path.join(modelpath,fname) for fname in 
              os.listdir(modelpath) if fname.endswith('.gmm')]
models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
speakers   = [fname.split("/")[-1].split(".gmm")[0] for fname 
              in gmm_files]


take = int(input("0 to test all the test files, 1 to select just one by name?"))
if take == 1:
    path = input("enter the filename:") 
    print("\ntesting audio file : ", path)
    sr,audio = read(source + path)
    winner = determine_speaker(audio,sr)
    print("\tdetected as - ", speakers[winner])
    time.sleep(1.0)
elif take == 0:
    source = "test_data/"
    os.chdir(source)       
    file_paths = sorted(glob.glob("*.wav"))
    os.chdir('..')
# Read the test directory and get the list of test audio files 
    for path in file_paths:   
            total_sample += 1.0
            path = path.strip()   
            print("\ntesting audio file: ", path)
            sr,audio = read(source + path)
            winner = determine_speaker(audio,sr)
            print("detected as - \t", speakers[winner])
            checker_name = path.split("_")[0]
            if speakers[winner] != checker_name:
                error += 1
            time.sleep(1.0)
    print(error, total_sample)
    accuracy = ((total_sample - error) / total_sample) * 100
    print ("Test run accuracy:", accuracy, "%")


0 to test all the test files, 1 to select just one by name?0

testing audio file:  dgovi_4.wav
detected as - 	 dgovi

testing audio file:  dgovi_5.wav
detected as - 	 dgovi

testing audio file:  dgovi_6.wav
detected as - 	 dgovi

testing audio file:  dgovi_7.wav
detected as - 	 dgovi

testing audio file:  dgovi_8.wav
detected as - 	 dgovi

testing audio file:  pnicoli_4.wav
detected as - 	 pnicoli

testing audio file:  pnicoli_5.wav
detected as - 	 pnicoli

testing audio file:  pnicoli_6.wav
detected as - 	 pnicoli

testing audio file:  pquadri_3.wav
detected as - 	 pquadri

testing audio file:  pquadri_4.wav
detected as - 	 pquadri

testing audio file:  pquadri_5.wav
detected as - 	 pnicoli

testing audio file:  pquadri_6.wav
detected as - 	 pquadri

testing audio file:  pquadri_7.wav
detected as - 	 pquadri

testing audio file:  pquadri_8.wav
detected as - 	 pnicoli
2 14.0
Test run accuracy: 85.71428571428571 %


## Test per un singolo audio in live

In [11]:
rec = sd.rec(int(duration * rec_rate), samplerate=rec_rate, channels=1, blocking=True)
print('Speaker identificato: ',speakers[determine_speaker(rec, rec_rate)])

pquadriscore = 	-33.30877127895754
dgoviscore = 	-32.40743716630698
pnicoliscore = 	-34.26579863299158
Speaker identificato:  dgovi
